In [1]:
!pip install tensorflow_text -q
!pip install transformers -q
!pip install -U sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text as text
import pandas as pd
from google.colab import drive
from transformers import BertTokenizer, TFBertModel
from sentence_transformers import SentenceTransformer

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
sentences = ["I ate dinner.", 
       "We had a three-course meal.", 
       "Brad came to dinner with us.",
       "He loves fish tacos.",
       "In the end, we all felt like we ate too much.",
       "We all agreed; it was a magnificent evening."]

sbert_model = SentenceTransformer('all-distilroberta-v1')
sentence_embeddings = sbert_model.encode(sentences)

# print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))
# print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])
query = "I had pizza and pasta"
query_vec = sbert_model.encode([query])[0]

In [5]:
sentence_embeddings.shape

(6, 768)

In [6]:
# query_vec

In [14]:
train_df = pd.read_csv("/content/drive/MyDrive/test_processed.csv")

In [15]:
train_df.columns

Index(['Unnamed: 0', 'PRODUCT_ID', 'TITLE', 'BULLET_POINTS', 'DESCRIPTION',
       'PRODUCT_TYPE_ID', 'T_BP', 'T_BP_D', 'T_D'],
      dtype='object')

In [16]:
train_df.head()

,Unnamed: 0,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,T_BP,T_BP_D,T_D
0,0,604373,manuel dheliogravure et de photogravure en rel...,NaN,NaN,6142,manuel dheliogravure et de photogravure en rel...,manuel dheliogravure et de photogravure en rel...,manuel dheliogravure et de photogravure en rel...
1,1,1729783,dcgaring microfiber throw blanket warm fuzzy p...,quality guarantee luxury cozy plush polyester ...,dcgaring throw blanket size chart w x l twin s...,1622,dcgaring microfiber throw blanket warm fuzzy p...,dcgaring microfiber throw blanket warm fuzzy p...,dcgaring microfiber throw blanket warm fuzzy p...
2,2,1871949,imatch auto part front license plate bracket t...,front license plate bracket make plasticdirect...,replacement follow vehicles2020 lexus nx300h 2...,7540,imatch auto parts front license plate bracket ...,imatch auto parts front license plate bracket ...,imatch auto parts front license plate bracket ...
3,3,1107571,pinmart gold plat excellence service 1 year aw...,available single item bulk pack select quantit...,excellence service lapel pin feature 3mm genui...,12442,pinmart gold plated excellence in service 1 ye...,pinmart gold plated excellence in service 1 ye...,pinmart gold plated excellence in service 1 ye...
4,4,624253,visual mathematics illustrate ti92 ti89,NaN,NaN,6318,visual mathematics illustrated by the ti92 and...,visual mathematics illustrated by the ti92 and...,visual mathematics illustrated by the ti92 and...


In [17]:
train_df['BULLET_POINTS'] = train_df['BULLET_POINTS'].replace(np.nan,'')
train_df['DESCRIPTION'] = train_df['DESCRIPTION'].replace(np.nan,'')
train_df['BP_D'] =  train_df['BULLET_POINTS'] + train_df['DESCRIPTION']

In [18]:
train_df['BULLET_POINTS'] = train_df['BULLET_POINTS'].replace('',np.nan)
train_df['DESCRIPTION'] = train_df['DESCRIPTION'].replace('',np.nan)

In [19]:
train_df = train_df.dropna(subset =['BP_D'])

In [13]:
import pandas as pd
from sklearn.utils import resample

# Load the dataset

# Separate the column with 19k entries
col = train_df['PRODUCT_TYPE_ID']

# Sample 50% of the column while preserving the distribution
sampled_col = resample(col, n_samples=0.25*len(col), random_state=42, stratify=col)

# Get the indices of the sampled entries
indices = sampled_col.index

# Sample the original dataframe based on the indices
sampled_df = train_df.loc[indices]

In [ ]:
sampled_df.to_csv('sampled_25_new_train_data.csv')

In [20]:
texts_list = list(train_df["BP_D"].astype(str))

In [21]:
sbert_model = SentenceTransformer('all-distilroberta-v1')

In [22]:
# texts_list1 = texts_list[:10000]

In [23]:
op_embed = []

In [24]:
texts_list

['',
 'quality guarantee luxury cozy plush polyester throw blanket keep couch bed spotless blanket design luxurious feel elegancesuper soft cozy super soft cozy luxury bed blanket perfect snuggle couch bed chill movie theater parkalso perfect personal gift occasionhome decor modern pattern digital print drape couch bed add artistic live room bed roomturns room invite space modern elegant design many design available match home decorblanket size 39 x 49 49 x 59 49 x 79 59 x 79 blanket throw size kid adults season choosesatisfaction guarantee provide 100 satisfaction service guarantee satisfy product offer full refunddcgaring throw blanket size chart w x l twin size 39lx49w full size 49lx59w queen size 49lx79w king size 59lx79w natural fabric make soft polyester fabric soft breathable microfiber fuzzy lightweight blanket keep year comfortable throw blanket suitable soft sofa chair couch picnic camp beach home decorative easy clean machine washable machine wash hand wash cold warm water b

In [ ]:
import time

s = time.time()

# for i in range(len(texts_list)):

#   sent_vec = sbert_model.encode([texts_list[i]])[0]
#   op_embed.append(sent_vec)

#   if i%10000 == 0:
#     print(i)

sentence_embeddings = sbert_model.encode(texts_list)

e = time.time()

print((e-s)/60, " mins")

In [ ]:
sentence_embeddings

array([[-0.01830868, -0.00736184, -0.02717656, ...,  0.01302763,
        -0.0191823 , -0.00358866],
       [-0.04203757, -0.06545904, -0.00536641, ...,  0.01887227,
        -0.03123164,  0.03991397],
       [-0.01830868, -0.00736184, -0.02717656, ...,  0.01302763,
        -0.0191823 , -0.00358866],
       ...,
       [ 0.00472405,  0.03121756, -0.01946985, ..., -0.04330277,
         0.02816974,  0.00012176],
       [-0.03436156, -0.00988398, -0.01631732, ...,  0.08223201,
        -0.05649769, -0.0316205 ],
       [-0.01830868, -0.00736184, -0.02717656, ...,  0.01302763,
        -0.0191823 , -0.00358866]], dtype=float32)

In [ ]:
np.save("sentence_embeddings_all-distilroberta-v1_BP_D_P1", sentence_embeddings)

In [ ]:
embeddings = np.load("sentence_embeddings_all-distilroberta-v1_BP_D_P1.npy")

In [ ]:
embeddings

array([[-0.01830868, -0.00736184, -0.02717656, ...,  0.01302763,
        -0.0191823 , -0.00358866],
       [-0.04203757, -0.06545904, -0.00536641, ...,  0.01887227,
        -0.03123164,  0.03991397],
       [-0.01830868, -0.00736184, -0.02717656, ...,  0.01302763,
        -0.0191823 , -0.00358866],
       ...,
       [ 0.00472405,  0.03121756, -0.01946985, ..., -0.04330277,
         0.02816974,  0.00012176],
       [-0.03436156, -0.00988398, -0.01631732, ...,  0.08223201,
        -0.05649769, -0.0316205 ],
       [-0.01830868, -0.00736184, -0.02717656, ...,  0.01302763,
        -0.0191823 , -0.00358866]], dtype=float32)

In [ ]:
from google.colab import files
files.download('sentence_embeddings_all-distilroberta-v1_BP_D_P2.npy') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>